In [1]:
from datasets import load_dataset, list_metrics, load_metric

blog_auth_ds = load_dataset('blog_authorship_corpus', download_mode="force_redownload")
print(blog_auth_ds['train'][0])

Generating train split:   0%|          | 0/689793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/37919 [00:00<?, ? examples/s]

Dataset blog_authorship_corpus downloaded and prepared to C:\Users\papam\.cache\huggingface\datasets\blog_authorship_corpus\blog_authorship_corpus\1.0.0\6f5d78241afd8313111956f877a57db7a0e9fc6718255dc85df0928197feb683. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'text': "Yeah, sorry for not writing for a whole there, but I've had a pretty busy weekend so far. I found out I have about 20 tests on Tuesday because my teachers are evil...but oh well. We had marching yesterday. It was cool. We stayed inside the whole time, and I got to play second part, so the music part will be really easy. I left early from marching at around 5:00, then went out to dinner with my friends (I made the plans before I knew about marching). So, dinner was fun. It was Kelly's birthday, so it was cool. Then we went to Kel's for a sleepover, and did bunches of neat stuff, and I went to sleep earlier than usual for a sleepover since I had to be at a Bar Mitzvah this morning. It was pretty neat. I have to do this project for CR, and we have to go to two different places of worship than our own. So, one is obviously a Jewish synagogue, and I think I'm going to go to a mosque for my other one, but who knows...so, this has been my only chance to write, and I actually have to

In [2]:
# Add column with the length of the texts
ds_with_length = blog_auth_ds.map(lambda x: {'length': len(x['text'].split())})
print(ds_with_length['train'][0]['length'])

  0%|          | 0/689793 [00:00<?, ?ex/s]

  0%|          | 0/37919 [00:00<?, ?ex/s]

209


In [3]:
# Tokenization
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
train_df = ds_with_length['train'].to_pandas()
train_df.head()

,text,date,gender,age,horoscope,job,length
0,"Yeah, sorry for not writing for a whole there,...","23,November,2002",female,17,Libra,Student,209
1,"Yeah, so today was ok, late arrival. I'm not i...","20,November,2002",female,17,Libra,Student,132
2,"Yay, Tuesday...no longer Monday! Whoopie! Plus...","19,November,2002",female,17,Libra,Student,628
3,RAR!,"18,November,2002",female,17,Libra,Student,1
4,"Thought- OK...so, I'm all for midgets and what...","18,November,2002",female,17,Libra,Student,30


In [5]:
val_df = ds_with_length['validation'].to_pandas()
val_df.head()

,text,date,gender,age,horoscope,job,length
0,"Well, everyone got up and going this morning. ...","31,May,2004",female,37,Leo,indUnk,107
1,My four-year old never stops talking. She'll ...,"29,May,2004",female,37,Leo,indUnk,48
2,"Actually it's not raining yet, but I bought 15...","28,May,2004",female,37,Leo,indUnk,96
3,Ha! Just set up my RSS feed - that is so easy!...,"28,May,2004",female,37,Leo,indUnk,67
4,"Oh, which just reminded me, we were talking ab...","28,May,2004",female,37,Leo,indUnk,141


In [6]:
train_df.shape, val_df.shape

((689793, 7), (37919, 7))

In [7]:
import pandas as pd

In [10]:
val_df['tokenized'] = val_df['text'].apply(tokenizer.tokenize).apply(" ".join)
val_df.head()

,text,date,gender,age,horoscope,job,length,tokenized
0,"Well, everyone got up and going this morning. ...","31,May,2004",female,37,Leo,indUnk,107,"well , everyone got up and going this morning ..."
1,My four-year old never stops talking. She'll ...,"29,May,2004",female,37,Leo,indUnk,48,my four - year old never stops talking . she '...
2,"Actually it's not raining yet, but I bought 15...","28,May,2004",female,37,Leo,indUnk,96,"actually it ' s not raining yet , but i bought..."
3,Ha! Just set up my RSS feed - that is so easy!...,"28,May,2004",female,37,Leo,indUnk,67,ha ! just set up my rs ##s feed - that is so e...
4,"Oh, which just reminded me, we were talking ab...","28,May,2004",female,37,Leo,indUnk,141,"oh , which just reminded me , we were talking ..."


In [11]:
train_df['tokenized'] = train_df['text'].apply(tokenizer.tokenize).apply(" ".join)
train_df.head()

,text,date,gender,age,horoscope,job,length,tokenized
0,"Yeah, sorry for not writing for a whole there,...","23,November,2002",female,17,Libra,Student,209,"yeah , sorry for not writing for a whole there..."
1,"Yeah, so today was ok, late arrival. I'm not i...","20,November,2002",female,17,Libra,Student,132,"yeah , so today was ok , late arrival . i ' m ..."
2,"Yay, Tuesday...no longer Monday! Whoopie! Plus...","19,November,2002",female,17,Libra,Student,628,"ya ##y , tuesday . . . no longer monday ! who ..."
3,RAR!,"18,November,2002",female,17,Libra,Student,1,ra ##r !
4,"Thought- OK...so, I'm all for midgets and what...","18,November,2002",female,17,Libra,Student,30,"thought - ok . . . so , i ' m all for mid ##ge..."


In [12]:
df = pd.concat([train_df, val_df], ignore_index=True)

In [13]:
df.shape

(727712, 8)

In [18]:
import csv

df.to_csv('blog_authorship_tokenized.csv', index=False, sep='|', encoding="utf-8", quoting=csv.QUOTE_NONE, escapechar='|')